In [ ]:
# importing libraries
import pandas as pd
import numpy as np

In [ ]:
# reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

In [ ]:
ratings

In [ ]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]

In [ ]:
print("The number of user:",n_users)
print("The number of n_items:",n_items)

 # Create pivot table for user and movie basedon ratings

In [ ]:
datama=ratings.pivot_table(index='user_id',columns='movie_id',values='rating')

In [ ]:
datama

In [ ]:
#Replace with nan with 0
data_matrix=datama.replace(np.nan,0)

In [ ]:
data_matrix

# Find Cosine Similatity for user and Item

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [ ]:
user_similarity 

# Using formula for user and item we are calcuating the score value

In [ ]:
import numpy as np
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        mean_user_rating = mean_user_rating.to_numpy().reshape(-1,1)
        ratings_diff = (ratings - mean_user_rating)
        pred = mean_user_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [ ]:
# prediction Table
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

In [ ]:
user_prediction

### As per User based filtering ,first have to find similarity between the input user and others

In [ ]:
#1. Select input user

input_user=34

In [ ]:
#2. Convert the user_sim table into DataFrame

user_sim_table=pd.DataFrame(user_similarity)

In [ ]:
#3. Find similarity user for 78 using cosine table

similar_input_user= user_sim_table[input_user].sort_values(ascending=True).head(5).index


In [ ]:
similar_input_user

In [ ]:
#4.Convert in to list

similar_user_input=list(similar_input_user)

In [ ]:
#5. Using similar_user_input,can select movie id from ratings table

similar_user_movieid_list=[]
for sim_user in similar_user_input:
    sim=list(ratings[ratings['user_id']==sim_user]['movie_id'])
    similar_user_movieid_list.append(sim)

In [ ]:
len(similar_user_movieid_list)

In [ ]:
#6. Convert all the list as single
import itertools
similar_user_movieid_single_list=list(itertools.chain.from_iterable(similar_user_movieid_list))

In [ ]:
len(similar_user_movieid_single_list)

#7. Unique movieid from the list

Unique_movieid_similar_user=set(similar_user_movieid_single_list)

In [ ]:
#7. Unique movieid from the list

Unique_movieid_similar_user=set(similar_user_movieid_single_list)



In [ ]:
len(Unique_movieid_similar_user)

In [ ]:
#8. Input user watched movie_list

input_user_watched_movieid=list(ratings[ratings['user_id']==input_user]['movie_id'].values)

In [ ]:
input_user_watched_movieid

In [ ]:
#9. Create a list which should have recom movieid to the input user

recom=[]
for per_id in Unique_movieid_similar_user:
    if(per_id in input_user_watched_movieid):
        pass
    else:
        recom.append(per_id)

In [ ]:
len(recom)

In [ ]:
sorted(recom)

In [ ]:
# Cross Checking 
sorted(Unique_movieid_similar_user)

In [ ]:
# Cross Checking
sorted(input_user_watched_movieid)

In [ ]:
# Checking the common movie list
list(set(Unique_movieid_similar_user)&set(input_user_watched_movieid))

In [ ]:
user_pred=pd.DataFrame(user_prediction)

In [ ]:
user_pred


In [ ]:
user_pred_Trans=user_pred.T

In [ ]:
user_pred_Trans[34]

In [ ]:
gg=user_pred_Trans[34]

In [ ]:
g=pd.DataFrame(gg)

In [ ]:
s=g.T

In [ ]:
s[187].values >=0.4

In [ ]:
# From recomd list select hightest rated film which would like by the user. Based on User prediction

highest_Rated=[]
input_user_pre=pd.DataFrame(user_pred_Trans[input_user])
input_user_pred=input_user_pre.T
for re in recom:
    value=input_user_pred[re].values
    if(value>=1):
        highest_Rated.append(re)


In [ ]:
# Now we give movieid respective movie list


i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,
encoding='latin-1')

In [ ]:
#Creating Movie List based on recom movieid
movie_title=[]
for movieid in highest_Rated:
    mov=items[items['movie id']==movieid]['movie title'].values
    movie_title.append(mov)

In [ ]:
movie_title

In [ ]:
#Converting into pure list
movie_title_list=[]
for m in movie_title:
    print(m)
    mv=list(m)
    movie_title_list.append(mv)

In [ ]:
#Converting into whole list
import itertools
Final_Recommend_movie=list(itertools.chain.from_iterable(movie_title_list))

In [ ]:
Final_Recommend_movie

In [ ]:
# Checking the common movie list
list(set(recom)&set(input_user_watched_movieid))

In [ ]:
def userbased(input_user,user_similarity,user_predictions,similar_user_count,thres):
    #Convert the user_sim table into DataFrame
    user_sim_table=pd.DataFrame(user_similarity)
    #Find similarity user for 78 using cosine table
    similar_input_user= user_sim_table[input_user].sort_values(ascending=True).head(similar_user_count).index
    #Convert in to list
    similar_user_input=list(similar_input_user) 
    #Using similar_user_input,can select movie id from ratings table
    similar_user_movieid_list=[]
    for sim_user in similar_user_input:
        sim=list(ratings[ratings['user_id']==sim_user]['movie_id'])
        similar_user_movieid_list.append(sim)
    #Converting as a whole list
    import itertools
    similar_user_movieid_single_list=list(itertools.chain.from_iterable(similar_user_movieid_list))
    #Unique movieid from the list
    Unique_movieid_similar_user=set(similar_user_movieid_single_list)
    #Input user watched movie_list
    input_user_watched_movieid=list(ratings[ratings['user_id']==input_user]['movie_id'].values)
    #Create a list which should have recom movieid to the input user
    recom=[]
    for per_id in Unique_movieid_similar_user:
        if(per_id in input_user_watched_movieid):
            pass
        else:
            recom.append(per_id)
    #From recommendation list selecting only hightest rated(predicted) value
    highest_Rated=[]
    user_pred=pd.DataFrame(user_prediction)
    user_pred_Trans=user_pred.T
    input_user_pre=pd.DataFrame(user_pred_Trans[input_user])
    input_user_pred=input_user_pre.T
    for re in recom:
        value=input_user_pred[re].values
        if(value>=thres):
            highest_Rated.append(re)
    i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
    'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
    'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
    items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')
    #Creating Movie List based on recom movieid
    movie_title=[]
    for movieid in highest_Rated:
        mov=items[items['movie id']==movieid]['movie title'].values
        movie_title.append(mov)
    #Converting into pure list
    movie_title_list=[]
    for m in movie_title:
        print(m)
        mv=list(m)
        movie_title_list.append(mv)
    #Converting into whole list
    import itertools
    Final_Recommend_movie=list(itertools.chain.from_iterable(movie_title_list))
    print("The common Movie in Recom & User:",list(set(recom)&set(input_user_watched_movieid)))
    return Final_Recommend_movie
    

In [ ]:
#def userbased(input_user,user_similarity,user_predictions,similar_user_count,similar_user_movieid_count,thres):
Recommended_movie=userbased(29,user_similarity,user_pred,2,0.8)

In [ ]:
len(Recommended_movie)